In [1]:
%%HTML
<style>.container { width:100% !important; }</style>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchmetrics.classification import BinaryAccuracy
from tqdm import tqdm
import json
import pandas as pd

# Select model

In [3]:
from Modeling import Meso4_09 as Curr_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Detect if GPU is available and use it
model = Curr_model(num_classes=1).to(device)  # Move model to GPU if available
model_name = str(model.__class__).split('.')[1][:-2]
print(f"{model_name}")
summary(model,model.input_shape)

Meso4_09
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 15, 240, 240]             420
       BatchNorm2d-2         [-1, 15, 240, 240]              30
         MaxPool2d-3         [-1, 15, 120, 120]               0
            Conv2d-4         [-1, 15, 120, 120]           5,640
       BatchNorm2d-5         [-1, 15, 120, 120]              30
         MaxPool2d-6           [-1, 15, 60, 60]               0
            Conv2d-7           [-1, 24, 60, 60]           9,024
       BatchNorm2d-8           [-1, 24, 60, 60]              48
         MaxPool2d-9           [-1, 24, 30, 30]               0
           Conv2d-10           [-1, 24, 30, 30]          14,424
      BatchNorm2d-11           [-1, 24, 30, 30]              48
        MaxPool2d-12             [-1, 24, 7, 7]               0
           Linear-13                   [-1, 15]          17,655
          Dropout-14          

# Dataset

In [4]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize(model.input_shape[1:]),  # Resize images 
    transforms.ToTensor(),        # Convert images to tensor
])

# Create dataset and dataloader
train_dataset = datasets.ImageFolder(root='../DataSet/real_vs_fake/real-vs-fake/train/', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)

val_dataset = datasets.ImageFolder(root='../DataSet/real_vs_fake/real-vs-fake/valid/', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)


In [5]:
# Configure the model for training , Set up criterion and optimizer
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
config ={
    "criterion": str(type(criterion)).split(".")[-1][:-2],
    "optimizer": str(type(optimizer)).split(".")[-1][:-2]
}
print(json.dumps(config, indent=4))

{
    "criterion": "BCEWithLogitsLoss",
    "optimizer": "Adam"
}


In [6]:
model_exists = False
config_templete = {
    "config":config,
    "last":{
        "epoch":0,
        "train_accuracy":0,
        "train_loss":0,
        "val_accuracy":0,
        "val_loss":900
    },
    "best":{
        "epoch":0,
        "train_accuracy":0,
        "train_loss":0,
        "val_accuracy":0,
        "val_loss":900
    }
}
all_models = json.load(open("config.json"))
if(0==len([i for i in all_models.keys() if model_name in i])):
    model_name1 = model_name+"_1"
    all_models[model_name1] = config_templete
    pd.DataFrame(columns=["epoch","train_accuracy","train_loss","val_accuracy","val_loss"]).to_csv("../History/"+model_name1+".csv",index=False)
    
else:

    dummy = [i for i in all_models if json.dumps(all_models[i]["config"])==json.dumps(config) and model_name in i]

    if(len(dummy)==0):
        model_name1 = model_name+"_"+str(max([i[:-2] for i in models]))
        all_models[model_name1] = config_templete
        pd.DataFrame(columns=["epoch","train_accuracy","train_loss","val_accuracy","val_loss"]).to_csv("../History/"+model_name1+".csv",index=False)

    elif(len(dummy)==1):
        model_name1 = dummy[0]
        model_exists = True
print(model_name1,json.dumps(all_models[model_name1],indent=3))

Meso4_09_1 {
   "config": {
      "criterion": "BCEWithLogitsLoss",
      "optimizer": "Adam"
   },
   "last": {
      "epoch": 0,
      "train_accuracy": 0,
      "train_loss": 0,
      "val_accuracy": 0,
      "val_loss": 900
   },
   "best": {
      "epoch": 0,
      "train_accuracy": 0,
      "train_loss": 0,
      "val_accuracy": 0,
      "val_loss": 900
   }
}


In [7]:
# load weights if available
# Load best model for inference or further training
best_model_path = "../Models/"+model_name1+".pth"
if(model_exists):
    model.load_state_dict(torch.load(best_model_path))
    model.to(device)  # Ensure model is on the correct dev

# update config at every epoch
def update_config(ls,mode):
    dct = dict(zip(["epoch","train_accuracy","train_loss","val_accuracy","val_loss"],ls))
    if(mode=="last"):
        pd.DataFrame([dct]).to_csv("../History/"+model_name1+".csv", mode='a',header=False,index=False)
    all_models[model_name1][mode]= dct
    with open("config.json", "w") as out_file:
        out_file.write(json.dumps(all_models,indent=4))

#  Validation

In [8]:
# Validation function
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    accuracy_metric = BinaryAccuracy().to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc='Validating', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            predictions = torch.sigmoid(outputs)
            correct += accuracy_metric(predictions, labels.int()).item()
            total += labels.size(0)
    accuracy = correct / len(dataloader)
    avg_loss = running_loss / len(dataloader)
    return avg_loss, accuracy

In [9]:
num_epochs = 10
start_epoch = all_models[model_name1]["last"]["epoch"]
# num_epochs = start_epoch+num_epochs
print(f"Epochs to train from     {start_epoch}   to   {num_epochs}")

Epochs to train from     0   to   10


# Training

In [10]:
# Training loop
best_val_loss = float('inf') if all_models[model_name1]["best"]["val_loss"]==None else all_models[model_name1]["best"]["val_loss"]# Initialize best validation loss to infinity
for epoch in range(start_epoch,num_epochs):  # number of epochs
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0
    accuracy_metric = BinaryAccuracy().to(device)
    train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True)
    for inputs, labels in train_loader_tqdm:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_accuracy += accuracy_metric(torch.sigmoid(outputs), labels.int()).item()
        train_loader_tqdm.set_postfix(loss=running_loss/len(train_loader_tqdm), accuracy=running_accuracy/len(train_loader_tqdm))
    
    train_loss = running_loss / len(train_loader)
    train_accuracy = running_accuracy / len(train_loader)
    val_loss, val_accuracy = validate(model, val_loader)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}')
    update_config(ls=[epoch+1,train_accuracy,train_loss,val_accuracy,val_loss], mode="last")
    # Save the best model if the validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        update_config(ls=[epoch+1,train_accuracy,train_loss,val_accuracy,val_loss], mode="best")
        print(f"Best model saved at epoch {epoch+1} with validation loss: {val_loss:.4f}")

        

# song_path ="../../../../../SYSTEM's/Byverse/Byverse  recovr/Mnx/Akshath - Nadaaniyan (Official Lyric Video)(M4A_128K).m4a"
# import IPython  
# IPython.display.Audio(song_path,autoplay=True)

Epoch 1/10: 100%|█| 3125/3125 [27:41<00:00,  1.88it/s, accuracy=0.621, loss=0.6


Epoch 1, Train Loss: 0.6036, Train Accuracy: 0.62, Val Loss: 0.4444, Val Accuracy: 0.82
Best model saved at epoch 1 with validation loss: 0.4444


Epoch 2/10: 100%|█| 3125/3125 [27:23<00:00,  1.90it/s, accuracy=0.683, loss=0.5


Epoch 2, Train Loss: 0.4999, Train Accuracy: 0.68, Val Loss: 0.3282, Val Accuracy: 0.89
Best model saved at epoch 2 with validation loss: 0.3282


Epoch 3/10: 100%|█| 3125/3125 [27:57<00:00,  1.86it/s, accuracy=0.706, loss=0.4


...

Epoch 8, Train Loss: 0.3889, Train Accuracy: 0.73, Val Loss: 0.1363, Val Accuracy: 0.96
Best model saved at epoch 8 with validation loss: 0.1363


Epoch 9/10: 100%|█| 3125/3125 [30:21<00:00,  1.72it/s, accuracy=0.734, loss=0.3


Epoch 9, Train Loss: 0.3880, Train Accuracy: 0.73, Val Loss: 0.1637, Val Accuracy: 0.94


Epoch 10/10: 100%|█| 3125/3125 [27:31<00:00,  1.89it/s, accuracy=0.736, loss=0.


Epoch 10, Train Loss: 0.3832, Train Accuracy: 0.74, Val Loss: 0.1230, Val Accuracy: 0.96
Best model saved at epoch 10 with validation loss: 0.1230


song_path ="D://SYSTEM's/Byverse/Byverse  recovr/Mnx/Akshath - Nadaaniyan (Official Lyric Video)(M4A_128K).m4a"
import IPython
import time
ply = IPython.display.Audio(song_path,autoplay=True)
temp = 0
while 1:
    IPython.display.display(ply)
    IPython.display.display(temp)
    time.sleep(2*60+52)
    temp += 2*60+52
    IPython.display.clear_output(wait=True)

In [ ]:
float('inf') 